# **📄 Document type classification baseline code**
> 문서 타입 분류 대회에 오신 여러분 환영합니다! 🎉     
> 아래 baseline에서는 ResNet 모델을 로드하여, 모델을 학습 및 예측 파일 생성하는 프로세스에 대해 알아보겠습니다.

## Contents
- Prepare Environments
- Import Library & Define Functions
- Hyper-parameters
- Load Data
- Train Model
- Inference & Save File


## 1. Prepare Environments

* 데이터 로드를 위한 구글 드라이브를 마운트합니다.
* 필요한 라이브러리를 설치합니다.

In [23]:
# 구글 드라이브 마운트, Colab을 이용하지 않는다면 패스해도 됩니다.
#from google.colab import drive
#rive.mount('/gdrive', force_remount=True)
#drive.mount('/content/drive')

In [24]:
# 구글 드라이브에 업로드된 대회 데이터를 압축 해제하고 로컬에 저장합니다.
#!tar -xvf drive/MyDrive/datasets_fin.tar > /dev/null

In [25]:
# !pip install -r requirements.txt 이미 서버에 설치되어 있음

In [26]:
# 필요한 라이브러리를 설치합니다.
# !pip install timm 이미 서버에 설치되어 있음

## 2. Import Library & Define Functions
* 학습 및 추론에 필요한 라이브러리를 로드합니다.
* 학습 및 추론에 필요한 함수와 클래스를 정의합니다.

In [27]:
import os
import time
import random

import timm
import torch
import albumentations as A
import pandas as pd
import numpy as np
import torch.nn as nn
from albumentations.pytorch import ToTensorV2
from torch.optim import Adam
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score

### 추가내용
from datetime import datetime       # 폴더이름에 날짜시간을 넣기 위한 라이브러리


In [28]:
### 
all_densenet_models = timm.list_models('*')
print(all_densenet_models)


['bat_resnext26ts', 'beit_base_patch16_224', 'beit_base_patch16_384', 'beit_large_patch16_224', 'beit_large_patch16_384', 'beit_large_patch16_512', 'beitv2_base_patch16_224', 'beitv2_large_patch16_224', 'botnet26t_256', 'botnet50ts_256', 'caformer_b36', 'caformer_m36', 'caformer_s18', 'caformer_s36', 'cait_m36_384', 'cait_m48_448', 'cait_s24_224', 'cait_s24_384', 'cait_s36_384', 'cait_xs24_384', 'cait_xxs24_224', 'cait_xxs24_384', 'cait_xxs36_224', 'cait_xxs36_384', 'coat_lite_medium', 'coat_lite_medium_384', 'coat_lite_mini', 'coat_lite_small', 'coat_lite_tiny', 'coat_mini', 'coat_small', 'coat_tiny', 'coatnet_0_224', 'coatnet_0_rw_224', 'coatnet_1_224', 'coatnet_1_rw_224', 'coatnet_2_224', 'coatnet_2_rw_224', 'coatnet_3_224', 'coatnet_3_rw_224', 'coatnet_4_224', 'coatnet_5_224', 'coatnet_bn_0_rw_224', 'coatnet_nano_cc_224', 'coatnet_nano_rw_224', 'coatnet_pico_rw_224', 'coatnet_rmlp_0_rw_224', 'coatnet_rmlp_1_rw2_224', 'coatnet_rmlp_1_rw_224', 'coatnet_rmlp_2_rw_224', 'coatnet_rmlp_2

In [29]:
# 시드를 고정합니다.
SEED = 42
os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.benchmark = True

In [30]:
# 데이터셋 클래스를 정의합니다.
class ImageDataset(Dataset):
    def __init__(self, csv, path, transform=None):
        self.df = pd.read_csv(csv).values
        self.path = path
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        name, target = self.df[idx]
        img = np.array(Image.open(os.path.join(self.path, name)))
        if self.transform:
            img = self.transform(image=img)['image']
        return img, target

In [31]:
# one epoch 학습을 위한 함수입니다.
def train_one_epoch(loader, model, optimizer, loss_fn, device):
    model.train()
    train_loss = 0
    preds_list = []
    targets_list = []

    pbar = tqdm(loader)
    for image, targets in pbar:
        image = image.to(device)
        targets = targets.to(device)

        model.zero_grad(set_to_none=True)

        preds = model(image)
        loss = loss_fn(preds, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())
        targets_list.extend(targets.detach().cpu().numpy())

        pbar.set_description(f"Loss: {loss.item():.4f}")

    train_loss /= len(loader)
    train_acc = accuracy_score(targets_list, preds_list)
    train_f1 = f1_score(targets_list, preds_list, average='macro')

    ret = {
        "train_loss": train_loss,
        "train_acc": train_acc,
        "train_f1": train_f1,
    }

    return ret

## 3. Hyper-parameters
* 학습 및 추론에 필요한 하이퍼파라미터들을 정의합니다.

In [32]:
# device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# data config
data_path = '/home/data/'
home_path = '/home/'

# model config
model_name = 'resnet34' # 'resnet50' 'efficientnet-b0', ...

# training config
img_size = 256
LR = 1e-3
EPOCHS = 1
BATCH_SIZE = 32
num_workers = 0

In [33]:
print(data_path + "data.csv")

/home/data/data.csv


## 4. Load Data
* 학습, 테스트 데이터셋과 로더를 정의합니다.

In [34]:
# augmentation을 위한 transform 코드
trn_transform = A.Compose([
    # 이미지 크기 조정
    A.Resize(height=img_size, width=img_size),
    # images normalization
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    # numpy 이미지나 PIL 이미지를 PyTorch 텐서로 변환
    ToTensorV2(),
])

# test image 변환을 위한 transform 코드
tst_transform = A.Compose([
    A.Resize(height=img_size, width=img_size),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2(),
])

"""# GPT 추천 
tst_transform = transforms.Compose([
    transforms.RandomRotation(degrees=45),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])"""

'# GPT 추천 \ntst_transform = transforms.Compose([\n    transforms.RandomRotation(degrees=45),\n    transforms.RandomHorizontalFlip(p=0.5),\n    transforms.RandomVerticalFlip(p=0.5),\n    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),\n    transforms.ToTensor(),\n    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])\n])'

In [35]:
# Dataset 정의
trn_dataset = ImageDataset(
    data_path + "train.csv",
    data_path + "train/",
    transform=trn_transform
)
tst_dataset = ImageDataset(
    data_path + "sample_submission.csv",
    data_path + "test/",
    transform=tst_transform
)
print(len(trn_dataset), len(tst_dataset))

1570 3140


In [36]:
# DataLoader 정의
trn_loader = DataLoader(
    trn_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=num_workers,
    pin_memory=True,
    drop_last=False
)
tst_loader = DataLoader(
    tst_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=0,
    pin_memory=True
)

## 5. Train Model
* 모델을 로드하고, 학습을 진행합니다.

In [37]:
# load model
model = timm.create_model(
    model_name,
    pretrained=True,
    num_classes=17
).to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=LR)

In [38]:
%%time 
#baseline 14.8s | 
#Loss: 2.5325: 100%|██████████| 50/50 [00:06<00:00,  7.65it/s]
#train_loss: 2.4731
#train_acc: 0.2554
#train_f1: 0.2238
#epoch: 0.0000

# 이게 기본값인가?
#Loss: 1.8296: 100%|██████████| 50/50 [00:09<00:00,  5.28it/s]
#train_loss: 1.6996
#train_acc: 0.5682
#train_f1: 0.5234
#epoch: 0.0000
#CPU times: user 22.1 s, sys: 624 ms, total: 22.7 s
#Wall time: 9.47 s

for epoch in range(EPOCHS):
    ret = train_one_epoch(trn_loader, model, optimizer, loss_fn, device=device)
    ret['epoch'] = epoch

    log = ""
    for k, v in ret.items():
      log += f"{k}: {v:.4f}\n"
    print(log)

Loss: 1.8017: 100%|██████████| 50/50 [00:08<00:00,  5.90it/s]


train_loss: 1.7012
train_acc: 0.5586
train_f1: 0.5100
epoch: 0.0000

CPU times: user 20.6 s, sys: 96.3 ms, total: 20.7 s
Wall time: 8.49 s


# 6. Inference & Save File
* 테스트 이미지에 대한 추론을 진행하고, 결과 파일을 저장합니다.

In [39]:
preds_list = []

model.eval()  # 모델을 평가 모드로 전환
for image, _ in tst_loader:  #tqdm(tst_loader): # 테스트 데이터 로더에서 이미지 가져오기
    image = image.to(device)  # 이미지를 GPU로 전송

    with torch.no_grad():  # 기울기 계산 비활성화
        preds = model(image)  # 예측 수행
    preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())  # 예측 결과 처리 및 저장

In [40]:
### test 데이터셋 분류 번호 붙이기
pred_df = pd.DataFrame(tst_dataset.df, columns=['ID', 'target'])
pred_df['target'] = preds_list

In [41]:
sample_submission_df = pd.read_csv(data_path + "sample_submission.csv")
assert (sample_submission_df['ID'] == pred_df['ID']).all()

In [42]:
current_time = datetime.now().strftime("%Y%m%d_%H%M%S")
print(current_time)

20240730_050400


In [43]:
pred_df.to_csv(home_path + "pred" + current_time + ".csv", index=False)

In [44]:
pred_df.head()

,ID,target
0,0008fdb22ddce0ce.jpg,2
1,00091bffdffd83de.jpg,11
2,00396fbc1f6cc21d.jpg,5
3,00471f8038d9c4b6.jpg,0
4,00901f504008d884.jpg,2
